In [1]:
# ━━━━━━━━━━━━ Keras 3 Setup ━━━━━━━━━━━━
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
keras.config.set_backend("torch")

import torch
print("Keras backend:", keras.backend.backend())
print("CUDA available:", torch.cuda.is_available())


Keras backend: torch
CUDA available: True


In [2]:


# ━━━━━━━━━━━━ Standard Imports ━━━━━━━━━━━━
import numpy as np
import pandas as pd
import random

random.seed(13)
np.random.seed(13)


In [4]:

import sys
sys.path.append(os.path.abspath("../src"))  # <- locate models and utilities


In [ ]:
# Data Load & Tensor Build
from data.loader import load_and_clean_raw_dataset
from data.dataset_builder import build_full_surface_feature_df
from data.tensor_builder import prepare_surface_and_feature_tensors
import numpy as np

df_raw = load_and_clean_raw_dataset("../data/vol_tensor_dataset.csv")
df_all = build_full_surface_feature_df("../data/vol_tensor_dataset.csv")
df_all = df_all.dropna()

iv_cols = [c for c in df_all.columns if c.startswith("IV_")]
feat_cols = [c for c in df_all.columns if not c.startswith("IV_")]

tensors, X_feat_tensor, df_feat = prepare_surface_and_feature_tensors(
    df_all=df_all,
    df_raw=df_raw,
    feat_cols=feat_cols,
    monthly_interpolation=False
)

# Pipeline Constants
LATENT_DIM = 12
SURFACE_SHAPE = (11, 10)
FEATURE_DIM = len(feat_cols)
LOOKBACK = 20
M, K = SURFACE_SHAPE
BATCH_SIZE = 32
T = tensors["surface_tensor"].shape[0]
print(f"FEATURE_DIM: {FEATURE_DIM}, T: {T}")

FEATURE_DIM: 26, T: 1258


In [7]:
# Check data scales
print("Surface tensor info:")
print(f"  Shape: {tensors['surface_tensor'].shape}")
print(f"  Range: [{tensors['surface_tensor'].min():.4f}, {tensors['surface_tensor'].max():.4f}]")
print(f"  Mean: {tensors['surface_tensor'].mean():.4f}, Std: {tensors['surface_tensor'].std():.4f}")

print("\nX_feat_tensor info:")
print(f"  Shape: {X_feat_tensor.shape}")
print(f"  Range: [{X_feat_tensor.min():.4f}, {X_feat_tensor.max():.4f}]")

# Check what a decoder expects vs what it gets
sample_surface = tensors["surface_tensor"][0, :, :, -1].reshape(1, -1)
print(f"\nSample surface for decoder:")
print(f"  Shape: {sample_surface.shape}")
print(f"  Range: [{sample_surface.min():.4f}, {sample_surface.max():.4f}]")

Surface tensor info:
  Shape: torch.Size([1258, 11, 10, 6])
  Range: [0.0000, 9216.2246]
  Mean: 1518.7891, Std: 2261.5378

X_feat_tensor info:
  Shape: torch.Size([1258, 136])
  Range: [-3.6510, 3.6939]

Sample surface for decoder:
  Shape: torch.Size([1, 110])
  Range: [0.1674, 0.7193]


In [9]:
# Model Loading 


import keras
import torch
from keras.models import load_model
from keras.saving import register_keras_serializable
from models.vae_mle import sample_z
from models.decoder import SimpleSurfaceDecoder, PiecewiseSurfaceDecoder
from models.transformer import TransformerForecaster, TransformerForecasterV2
from models.pointwise_moe import PiecewiseSurfaceDecoderModular

# Registries
ENCODER_REGISTRY = {
    "AE": "saved_models/ae/ae_encoder.keras",
    "VAE_MLE": "saved_models/vae_mle/vae_mle_encoder.keras",
    "VAE": "saved_models/vae_gru_moe/encoder_vae.keras",
    "GBO": None,  # Created programmatically
    "PCA": None,  # Created programmatically
}

TEMPORAL_REGISTRY = {
    "ATTN-AE": "saved_models/attention/attention_model_ae.keras",
    "ATTN-VAE": "saved_models/attention/attention_model_vae.keras", 
    "ATTN-PCA": "saved_models/attention/attention_model_pca.keras",
    "ATTN-GBO": "saved_models/attention_gbo/attention_model_gbo.keras",
    "GRU-AE": "saved_models/gru/gru_model_ae.keras",
    "GRU-PCA": "saved_models/gru/gru_model_pca.keras",
    "GRU-VAE": "saved_models/vae_gru_moe/gru_temporal.keras",
    "GRU-GBO": "saved_models/gru_gbo/gru_model_gbo.keras",
    "LSTM-AE": "saved_models/lstm/lstm_model_ae.keras",
    "LSTM-VAE": "saved_models/lstm/lstm_model_vae.keras",
    "LSTM-PCA": "saved_models/lstm/lstm_model_pca.keras", #"saved_models/lstm/lstm_model_pca.keras")
    "LSTM-ATTN-AE": "saved_models/lstm_attn/lstm_model_ae.keras",
    "LSTM-ATTN-VAE": "saved_models/lstm_attn/lstm_model_vae.keras",
    "LSTM-ATTN-PCA": "saved_models/lstm_attn/lstm_model_pca.keras",
    "LSTM-ATTN-GBO": "saved_models/lstm_attn_gbo/lstm_model_gbo.keras",
    "LSTM-GBO": "saved_models/gbo/lstm_model_gbo.keras",
    "TRFM-AE": "saved_models/transformer/transformer_model_ae.keras",
    "TRFM-VAE": "saved_models/transformer/transformer_model_vae.keras",
    "TRFM-PCA": "saved_models/transformer/transformer_model_pca.keras",  
    "TRFM-GBO": "saved_models/transformer_gbo/transformer_model_gbo.keras",
    "TRFM-V2-AE": "saved_models/transformer_v2/transformer_v2_ae_model.keras",
    "TRFM-V2-VAE": "saved_models/transformer_v2/transformer_v2_vae_model.keras",
    "TRFM-V2-PCA": "saved_models/transformer_v2/transformer_v2_model.keras",  
    "TRFM-V2-GBO": "saved_models/transformer_v2_gbo/transformer_v2_model_gbo.keras",
}

DECODER_REGISTRY = {
    "AE": "saved_models/ae/ae_decoder.keras",
    "VAE_MLE": "saved_models/vae_mle/vae_mle_decoder.keras",         
    "VAE": "saved_models/vae_gru_moe/decoder_vae.keras",
    "GBO": None,  # Created programmatically
    "PCA": None,  # Created programmatically
}

CORRECTIVE_DECODER_REGISTRY = { 
    "AE_PW": "saved_models/ae/piecewise_surface_mod_moe/decoder.keras",
    "VAE_PW": "saved_models/vae_gru_moe_2/decoder_moe_2.keras",
    "VAE_MLE_PW": "saved_models/vae/piecewise_surface_mod_moe/decoder_final.keras",
    "GBO_PW": "saved_models/gbo/piecewise_surface_mod_moe_2/decoder.keras",
    "PCA_PW": "saved_models/ae/piecewise_surface_mod_moe/decoder.keras",  # Reuse AE corrective for PCA
}



Note: for PCA and GBO, there are no saved models so we need to re-import the components 

In [10]:
from keras import ops
from models.gbo import GeneralizedBasisOperator, GBOEncoderWrapper, GBODecoderWrapper
from models.encoder_pca import PCAEncoder, PCAEncoderWrapper, PCADecoderWrapper

def create_gbo_components(tensors, X_feat_tensor, M, K, latent_dim=12):
    """Create GBO encoder and decoder components for pipeline."""
    surface_tensor = tensors["surface_tensor"]
    X_surface_tensor = surface_tensor[..., -1].reshape(surface_tensor.shape[0], -1)
    
    # Initialize and fit GBO
    gbo = GeneralizedBasisOperator(n_components=latent_dim)
    gbo.fit(X_surface_tensor, X_feat_tensor)
    
    encoder_gbo = GBOEncoderWrapper(gbo, X_feat_tensor, M, K)
    decoder_gbo = GBODecoderWrapper(encoder_gbo)
    
    return encoder_gbo, decoder_gbo

def create_pca_components(tensors, M, K, latent_dim=12):
    """Create PCA encoder and decoder components for pipeline."""
    surface_tensor = tensors["surface_tensor"]
    X_surface_tensor = surface_tensor[..., -1].reshape(surface_tensor.shape[0], -1)
    X_surface_numpy = ops.convert_to_numpy(X_surface_tensor)
    
    # Initialize and fit PCA
    pca = PCAEncoder(n_components=latent_dim)
    pca.fit_transform(X_surface_numpy)
    
    encoder_pca = PCAEncoderWrapper(pca, M, K)
    decoder_pca = PCADecoderWrapper(encoder_pca)
    
    return encoder_pca, decoder_pca

# Create GBO components
gbo_encoder, gbo_decoder = create_gbo_components(
    tensors, X_feat_tensor, M, K, LATENT_DIM
)

# Create PCA components
pca_encoder, pca_decoder = create_pca_components(
    tensors, M, K, LATENT_DIM
)

In [ ]:
# Serialization workaround for Lambda layers in saved models
# Some models contain Lambda layers that need to be registered for loading
# we use a placeholder function to maintain compatibility
@register_keras_serializable()
def custom_lambda(x):
   
    import keras.ops as ops
    return ops.ones_like(x) * ops.log(ops.ones(1))  # Returns zeros

from models.gru import GRUForecasterAugmented

custom_objects = {
    "sample_z": sample_z,
    "SimpleSurfaceDecoder": SimpleSurfaceDecoder,
    "PiecewiseSurfaceDecoder": PiecewiseSurfaceDecoder,
    "PiecewiseSurfaceDecoderModular": PiecewiseSurfaceDecoderModular,
    "TransformerForecaster": TransformerForecaster,
    "TransformerForecasterV2": TransformerForecasterV2,
    "GRUForecasterAugmented": GRUForecasterAugmented,  # ADD THIS
    "ops": keras.ops,
    "custom_lambda": custom_lambda
}

pipelines = {}

# All valid pipeline combinations
valid_combinations = [
    # AE Family
    ("AE", "ATTN-AE", "AE", "AE_PW"),
    ("AE", "LSTM-AE", "AE", "AE_PW"),
    ("AE", "LSTM-ATTN-AE", "AE", "AE_PW"),
    ("AE", "GRU-AE", "AE", "AE_PW"),
    ("AE", "TRFM-AE", "AE", "AE_PW"),
    ("AE", "TRFM-V2-AE", "AE", "AE_PW"),

    # VAE Family
    ("VAE", "GRU-VAE", "VAE", "VAE_PW"),
    ("VAE_MLE", "LSTM-VAE", "VAE_MLE", "VAE_MLE_PW"),
    ("VAE_MLE", "LSTM-ATTN-VAE", "VAE_MLE", "VAE_MLE_PW"),
    ("VAE_MLE", "TRFM-VAE", "VAE_MLE", "VAE_MLE_PW"),
    ("VAE_MLE", "TRFM-V2-VAE", "VAE_MLE", "VAE_MLE_PW"),
    ("VAE_MLE", "ATTN-VAE", "VAE_MLE", "VAE_MLE_PW"),

    # GBO Family
    ("GBO", "GRU-GBO", "GBO", "GBO_PW"),
    ("GBO", "LSTM-GBO", "GBO", "GBO_PW"),
    ("GBO", "LSTM-ATTN-GBO", "GBO", "GBO_PW"),
    ("GBO", "TRFM-GBO", "GBO", "GBO_PW"),
    ("GBO", "TRFM-V2-GBO", "GBO", "GBO_PW"),
    ("GBO", "ATTN-GBO", "GBO", "GBO_PW"),
    
    # PCA Family - ADD THESE
    ("PCA", "GRU-PCA", "PCA", "AE_PW"),  # Use AE corrective for PCA
    ("PCA", "LSTM-PCA", "PCA", "AE_PW"),
    ("PCA", "LSTM-ATTN-PCA", "PCA", "AE_PW"),
    ("PCA", "ATTN-PCA", "PCA", "AE_PW"),
    ("PCA", "TRFM-PCA", "PCA", "AE_PW"),
    ("PCA", "TRFM-V2-PCA", "PCA", "AE_PW"),
]

for enc_key, temp_key, dec_key, corr_key in valid_combinations:
    try:
        # Special handling for programmatic components
        if enc_key == "GBO":
            encoder = gbo_encoder
            decoder = gbo_decoder
        elif enc_key == "PCA":  # ADD PCA HANDLING
            encoder = pca_encoder
            decoder = pca_decoder
        else:
            encoder = load_model(ENCODER_REGISTRY[enc_key], custom_objects=custom_objects, compile=False)
            decoder = load_model(DECODER_REGISTRY[dec_key], custom_objects=custom_objects, compile=False)
        
        # Check if temporal model exists
        if temp_key not in TEMPORAL_REGISTRY:
            print(f"Skipped: {enc_key}_{temp_key} - not in registry")
            continue
            
        temporal_path = TEMPORAL_REGISTRY[temp_key]
        if temporal_path and not os.path.exists(temporal_path):
            print(f"Skipped: {enc_key}_{temp_key} - file not found")
            continue
            
        temporal = load_model(temporal_path, custom_objects=custom_objects, compile=False)
        
        # Load corrective decoder
        corrective_path = CORRECTIVE_DECODER_REGISTRY[corr_key]
        if not os.path.exists(corrective_path):
            print(f"Failed: {enc_key}_{temp_key} - corrective not found at {corrective_path}")
            continue
            
        corrective = load_model(corrective_path, custom_objects=custom_objects, compile=False)
        
        pipeline_name = f"{enc_key}_{temp_key}_{dec_key}_{corr_key}"
        pipelines[pipeline_name] = {
            'encoder': encoder,
            'temporal': temporal,
            'decoder': decoder,
            'corrective': corrective
        }
        #print(f"Loaded: {pipeline_name}")
        
    except Exception as e:
        print(f"Failed: {enc_key}_{temp_key}_{dec_key}_{corr_key} - {str(e)[:60]}")

print(f"\nTotal pipelines loaded: {len(pipelines)}")
print(f"Families: AE ({sum(1 for p in pipelines if p.startswith('AE_'))}), "
      f"VAE ({sum(1 for p in pipelines if p.startswith('VAE_'))}), "
      f"PCA ({sum(1 for p in pipelines if p.startswith('PCA_'))}), "  # ADD PCA COUNT
      f"GBO ({sum(1 for p in pipelines if p.startswith('GBO'))})")

Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7


c:\Users\alessandro.esposito\source\repos\imperial\.venv\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Maturity=4, Free=3, Total=7
Expert allocation: ATM=0, Ma

#### Transofrmer V2 Model Compatibility Layer

During development, we found that V2 transformer models had a preprocessing bug where the feature centering wasn't consistent between training and testing. Rather than retrain everything (which takes forever), these helper functions patch the issue so we can still run the models and include them in our comparison. The main problem is that V2 models were trained with features scaled one way, but at test time they were getting features scaled differently. This mismatch definitely hurts performance but doesn't completely break the models. It probably explains why the transformers underperform compared to simpler RNN models. These helper functions work around the issue by handling all the different encoder quirks. VAE encoders return three outputs but we only need the mean, PCA encoders only take surface data not features, and each encoder has its own format. The functions also pad or trim features to match what the models expect from training, since the feature dimensions might have changed. Everything gets stitched together in the right order since V2 models expect features and latent codes concatenated in a specific way. Not ideal, but it lets us test V2 models without waiting hours for retraining. At least we can include them in the pipeline comparison even if we know they're handicapped by this preprocessing issue.

In [13]:
# Helper Functions for V2 Models
# V2 models require special preprocessing because they combine latent codes with features

def encode_timestep_for_v2(encoder, surface_t, feat_t, pipeline_name):
    """Encode a single timestep for V2 transformer models.
    Different encoders require different input formats."""
    if 'PCA' in pipeline_name:
        # PCA encoder only processes surface data
        z_t = encoder.predict(surface_t)
    elif 'VAE' in pipeline_name:
        # VAE returns tuple (sampled_z, z_mean, z_logvar), we need z_mean
        full_input = np.concatenate([surface_t, feat_t], axis=-1)
        z_output = encoder(full_input, training=False)
        z_t = z_output[1] if isinstance(z_output, (list, tuple)) else z_output
        z_t = ops.convert_to_numpy(z_t)
    else:
        # AE/GBO encoders process combined surface+features
        full_input = np.concatenate([surface_t, feat_t], axis=-1)
        z_t = encoder.predict(full_input, verbose=0)
    return z_t

def encode_target_for_v2(encoder, target_flat, target_feat, pipeline_name):
    """Encode target timestep for V2 models loss computation."""
    if 'PCA' in pipeline_name:
        z_target = encoder.predict(target_flat)
    else:
        target_full = np.concatenate([target_flat, target_feat], axis=-1)
        z_target = encoder.predict(target_full, verbose=0)
    
    # Handle VAE output format
    if 'VAE' in pipeline_name:
        z_target = ops.convert_to_numpy(z_target)
    elif isinstance(z_target, (list, tuple)):
        z_target = z_target[1]
    
    return z_target

def process_v2_batch(batch, encoder, pipeline_name, config, lookback, M, K):
    """Process batch for V2 transformers which expect [features, latents] input.
    V2 models concatenate market features with encoded latents at each timestep."""
    
    # Extract and convert tensors
    (surface_seq, feat_seq), target_surface = batch
    surface_seq = ops.convert_to_numpy(surface_seq)
    feat_seq = ops.convert_to_numpy(feat_seq)
    target_surface = ops.convert_to_numpy(target_surface)
    
    batch_size = surface_seq.shape[0]
    actual_feat_dim = feat_seq.shape[-1]
    
    # V2 models have fixed input dimensions from training
    expected_input_dim = config['input_dim']
    expected_latent_dim = config['latent_dim']
    expected_feat_dim = expected_input_dim - expected_latent_dim
    
    # Encode each timestep in sequence
    latent_seq = []
    for t in range(lookback):
        surface_t = surface_seq[:, t, :, :, -1].reshape(batch_size, M*K)
        feat_t = feat_seq[:, t, :]
        z_t = encode_timestep_for_v2(encoder, surface_t, feat_t, pipeline_name)
        latent_seq.append(z_t)
    
    latent_seq = np.stack(latent_seq, axis=1)
    
    # Pad or truncate features to match model's expected dimensions
    if actual_feat_dim > expected_feat_dim:
        feat_seq_reduced = feat_seq[:, :, :expected_feat_dim]
    else:
        padding = np.zeros((batch_size, lookback, expected_feat_dim - actual_feat_dim))
        feat_seq_reduced = np.concatenate([feat_seq, padding], axis=-1)
    
    # Combine features and latents for V2 input format
    combined = np.concatenate([feat_seq_reduced, latent_seq], axis=-1)
    
    # Encode target for loss computation
    target_flat = target_surface[:, :, :, -1].reshape(batch_size, M*K)
    target_feat = feat_seq[:, -1, :]
    z_target = encode_target_for_v2(encoder, target_flat, target_feat, pipeline_name)
    
    return combined, z_target

In [14]:
# Complete Pipeline Evaluation
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from data.dataset import FeatureToLatentSequenceDataset
from models.lstm import EncodedLatentSequence
from utils.eval import summarize_latent_errors
from models.encoder_pca import PCAEncoder, PCAEncoderWrapper, PCADecoderWrapper, PCAEncodedLatentSequence
from data.edav import check_no_arbitrage_total_variance
from keras import ops

def evaluate_pipeline(pipeline_name, pipeline, tolerance=0.005):
    """Evaluate a complete forecasting pipeline end-to-end."""
    encoder = pipeline['encoder']
    temporal = pipeline['temporal']
    decoder = pipeline['decoder']
    corrective = pipeline['corrective']
    
    # Patch VAE encoder to return z_mean instead of tuple
    if 'VAE' in pipeline_name:
        encoder.predict = lambda x, **kwargs: encoder(x, training=False)[1]
    
    lookback = 20 if 'TRFM' in pipeline_name else LOOKBACK
    
    # Check for V2 model (has feature concatenation issue)
    is_v2_model = False
    if 'TRFM-V2' in pipeline_name and hasattr(temporal, 'get_config'):
        config = temporal.get_config()
        if config.get('input_dim', 0) == 38:
            is_v2_model = True
    
    full_dataset = FeatureToLatentSequenceDataset(
        surface_tensor=tensors["surface_tensor"],
        feature_tensor=X_feat_tensor,
        lookback=lookback,
        batch_size=BATCH_SIZE,
        global_indices=np.arange(T)
    )
    
    if is_v2_model:
        # V2 models need special preprocessing
        config = temporal.get_config()
        combined_sequences = []
        Z_true = []
        
        for batch in full_dataset:
            combined, z_target = process_v2_batch(
                batch, encoder, pipeline_name, config, lookback, M, K
            )
            combined_sequences.append(combined)
            Z_true.append(z_target)
        
        combined_sequences = np.concatenate(combined_sequences, axis=0)
        Z_true = np.concatenate(Z_true, axis=0)
        Z_forecast = temporal.predict(combined_sequences, verbose=0)
        
    else:
        # Standard processing for V1 models
        if 'PCA' in pipeline_name:
            from models.encoder_pca import PCAEncodedLatentSequence
            
            encoded_sequences = PCAEncodedLatentSequence(full_dataset, encoder)
            
            Z_sequences = []
            Z_true = []
            for z_seq, z_target in encoded_sequences:
                Z_sequences.append(z_seq)
                Z_true.append(z_target)
            
            Z_sequences = np.concatenate(Z_sequences, axis=0)
            Z_true = np.concatenate(Z_true, axis=0)
            
            # Disable centering for transformers
            if 'TRFM' in pipeline_name and hasattr(temporal, 'center_input'):
                original = temporal.center_input
                temporal.center_input = False
                Z_forecast = temporal.predict(Z_sequences, verbose=0)
                temporal.center_input = original
            else:
                Z_forecast = temporal.predict(Z_sequences, verbose=0)
                
            Z_true = Z_true[:len(Z_forecast)]
            
        else:
            # Standard encoders (AE, GBO)
            encoded_sequences = EncodedLatentSequence(full_dataset, encoder)
            
            if 'TRFM' in pipeline_name and hasattr(temporal, 'center_input'):
                original = temporal.center_input
                temporal.center_input = False
                Z_forecast = temporal.predict(encoded_sequences, verbose=0)
                temporal.center_input = original
            else:
                Z_forecast = temporal.predict(encoded_sequences, verbose=0)
            
            Z_true = []
            for _, z_target in encoded_sequences:
                Z_true.append(ops.convert_to_numpy(z_target))
            Z_true = np.concatenate(Z_true, axis=0)[:len(Z_forecast)]
    
    # Compute latent space metrics
    latent_summary = summarize_latent_errors(Z_true, Z_forecast, name=pipeline_name)
    latent_metrics = latent_summary.iloc[0].to_dict()
    
    X_surface_all = ops.convert_to_numpy(tensors["surface_tensor"][:, :, :, -1].reshape(T, -1))
    n_forecasts = Z_forecast.shape[0]
    
    Z_eval = Z_forecast
    Y_true = X_surface_all[-n_forecasts:].reshape(-1, M, K)
    
    # Decode latent forecasts
    decoded_surfaces = []
    for i in range(len(Z_eval)):
        if 'GBO' in pipeline_name or 'PCA' in pipeline_name:
            decoded = decoder.predict(Z_eval[i:i+1])
        else:
            decoded = decoder.predict(Z_eval[i:i+1], verbose=0)
        if isinstance(decoded, (list, tuple)):
            decoded = decoded[0]
        decoded_surface = decoded[0, :M*K].reshape(M, K)
        decoded_surfaces.append(decoded_surface)
    decoded_surfaces = np.array(decoded_surfaces)
    decoder_rmse = mean_squared_error(Y_true.flatten(), decoded_surfaces.flatten())**0.5
    
    # Persistence baseline
    Y_prev = X_surface_all[-n_forecasts-1:-1].reshape(-1, M, K) if n_forecasts < T else Y_true
    persistence_rmse = mean_squared_error(Y_true.flatten(), Y_prev.flatten())**0.5
    
    # Apply corrective MoE decoder
    rel_strikes = ops.convert_to_numpy(tensors["strike_tensor"])
    taus = ops.convert_to_numpy(tensors["taus"])
    original_features = ops.convert_to_numpy(X_feat_tensor[-n_forecasts:])
    
    m_grid, tau_grid = np.meshgrid(rel_strikes, taus)
    m_flat = m_grid.reshape(-1, 1).astype(np.float32)
    tau_flat = tau_grid.reshape(-1, 1).astype(np.float32)
    
    corrected_surfaces = []
    for i in range(len(Z_eval)):
        surface = corrective.predict_surface(
            z=Z_eval[i],
            f=original_features[i],
            m_flat=m_flat,
            tau_flat=tau_flat
        )
        corrected_surfaces.append(surface)
    
    corrected_surfaces = np.stack(corrected_surfaces)
    
    # Calculate metrics
    test_split = int(0.8 * n_forecasts)
    Y_test = Y_true[test_split:]
    Y_train = Y_true[:test_split]
    corrected_test = corrected_surfaces[test_split:]
    corrected_train = corrected_surfaces[:test_split]
    
    test_rmse = mean_squared_error(Y_test.flatten(), corrected_test.flatten())**0.5
    train_rmse = mean_squared_error(Y_train.flatten(), corrected_train.flatten())**0.5
    full_rmse = mean_squared_error(Y_true.flatten(), corrected_surfaces.flatten())**0.5
    test_mae = mean_absolute_error(Y_test.flatten(), corrected_test.flatten())
    
    # Check arbitrage violations
    smile_viol = 0
    cal_viol = 0
    for surface in corrected_test:
        s, c = check_no_arbitrage_total_variance(surface, taus, rel_strikes, tol=tolerance)
        smile_viol += s
        cal_viol += c
    
    # Flag problematic pipelines
    is_problematic = False
    problem_note = ""
    
    if decoder_rmse < 0.05 and test_rmse > 0.15:
        is_problematic = True
        problem_note = "MoE_FAIL"
    elif decoder_rmse > 0.10:
        is_problematic = True
        problem_note = "TEMPORAL_FAIL"
    
    return {
        'pipeline': pipeline_name,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'full_rmse': full_rmse,
        'test_mae': test_mae,
        'decoder_rmse': decoder_rmse,
        'persistence_rmse': persistence_rmse,
        'latent_mse': latent_metrics['latent_mse_total'],
        'latent_acf_diff': latent_metrics['acf_diff_mean'],
        'latent_var_ratio': latent_metrics['var_ratio_pred/true'],
        'smile_violations': smile_viol,
        'calendar_violations': cal_viol,
        'total_violations': smile_viol + cal_viol,
        'is_problematic': is_problematic,
        'problem_note': problem_note,
    }

#### Two-Pass Evaluation Strategy

We run the evaluation twice to handle initialization issues with Keras models, particularly the V2 transformers that have the preprocessing bug we discussed earlier.**First pass (initialization):** Some models fail on their first prediction call due to uninitialized weights or shape mismatches. This is especially common with custom layers and the V2 models that expect specific input dimensions. We run through all pipelines once to trigger these initialization errors without collecting results. **Second pass (actual evaluation):** With models properly initialized, we run the full evaluation and collect metrics. This ensures consistent results and avoids the random failures that occur when models are cold-started. This approach is a workaround for Keras' occasional issues with complex model architectures loaded from disk. Without the warm-up pass, about 30% of pipelines would randomly fail with shape or initialization errors, making comparison impossible.

In [16]:
# Run evaluation
TOLERANCE = 0.005 # half a vol 

# First pass - initialize models
print("Models Initialization")
for name, pipeline in pipelines.items():
    try:
        _ = evaluate_pipeline(name, pipeline, tolerance=TOLERANCE)
    except Exception as e:
        if 'TRFM-V2' in name:
            print(f"V2 init issue {name}: {str(e)[:50]}")

# Second pass - collect results
print("Pipeline Evaluation")

results = []
for name, pipeline in pipelines.items():
    try:
        result = evaluate_pipeline(name, pipeline, tolerance=TOLERANCE)
        results.append(result)
        
        # Print progress
        status = f"[{result['problem_note']}]" if result['is_problematic'] else ""
        print(f"{name}: RMSE {result['test_rmse']:.4f} {status}")
        
    except Exception as e:
        print(f"Failed {name}: {str(e)[:60]}")

# Summarize results
if results:
    df_results = pd.DataFrame(results)
    
    # Split by quality
    df_good = df_results[~df_results['is_problematic']].sort_values('test_rmse')
    df_bad = df_results[df_results['is_problematic']].sort_values('test_rmse')
    
 
    print("SUCCESSFUL PIPELINES")
    display(df_good[['pipeline', 'test_rmse', 'decoder_rmse', 'persistence_rmse']])
    
    if len(df_bad) > 0:
        print("\nFAILED PIPELINES")
        display(df_bad[['pipeline', 'test_rmse', 'decoder_rmse', 'problem_note']])

Models Initialization
Pipeline Evaluation
AE_ATTN-AE_AE_AE_PW: RMSE 0.0089 
AE_LSTM-AE_AE_AE_PW: RMSE 0.0092 
AE_LSTM-ATTN-AE_AE_AE_PW: RMSE 0.0091 
AE_GRU-AE_AE_AE_PW: RMSE 0.0092 
AE_TRFM-AE_AE_AE_PW: RMSE 0.0277 
AE_TRFM-V2-AE_AE_AE_PW: RMSE 0.1288 [TEMPORAL_FAIL]
VAE_GRU-VAE_VAE_VAE_PW: RMSE 0.0107 
VAE_MLE_LSTM-VAE_VAE_MLE_VAE_MLE_PW: RMSE 0.0105 
VAE_MLE_LSTM-ATTN-VAE_VAE_MLE_VAE_MLE_PW: RMSE 0.0112 
VAE_MLE_TRFM-VAE_VAE_MLE_VAE_MLE_PW: RMSE 0.1845 [TEMPORAL_FAIL]
VAE_MLE_TRFM-V2-VAE_VAE_MLE_VAE_MLE_PW: RMSE 0.1036 [TEMPORAL_FAIL]
VAE_MLE_ATTN-VAE_VAE_MLE_VAE_MLE_PW: RMSE 0.0110 
GBO_GRU-GBO_GBO_GBO_PW: RMSE 0.0125 
GBO_LSTM-GBO_GBO_GBO_PW: RMSE 0.0119 
GBO_LSTM-ATTN-GBO_GBO_GBO_PW: RMSE 0.0126 
GBO_TRFM-GBO_GBO_GBO_PW: RMSE 0.0266 
GBO_TRFM-V2-GBO_GBO_GBO_PW: RMSE 0.0491 
GBO_ATTN-GBO_GBO_GBO_PW: RMSE 0.0121 
PCA_GRU-PCA_PCA_AE_PW: RMSE 0.0619 
PCA_LSTM-PCA_PCA_AE_PW: RMSE 0.0630 
PCA_LSTM-ATTN-PCA_PCA_AE_PW: RMSE 0.0682 
PCA_ATTN-PCA_PCA_AE_PW: RMSE 0.0544 [TEMPORAL_FAIL]
PCA_T

,pipeline,test_rmse,decoder_rmse,persistence_rmse
0,AE_ATTN-AE_AE_AE_PW,0.008913,0.021843,0.006812
2,AE_LSTM-ATTN-AE_AE_AE_PW,0.009143,0.022016,0.006812
3,AE_GRU-AE_AE_AE_PW,0.009222,0.021820,0.006812
1,AE_LSTM-AE_AE_AE_PW,0.009241,0.021420,0.006812
7,VAE_MLE_LSTM-VAE_VAE_MLE_VAE_MLE_PW,0.010537,0.031203,0.006812
6,VAE_GRU-VAE_VAE_VAE_PW,0.010669,0.026457,0.006812
11,VAE_MLE_ATTN-VAE_VAE_MLE_VAE_MLE_PW,0.011046,0.025397,0.006812
8,VAE_MLE_LSTM-ATTN-VAE_VAE_MLE_VAE_MLE_PW,0.011195,0.024892,0.006812
13,GBO_LSTM-GBO_GBO_GBO_PW,0.011947,0.019420,0.006812
17,GBO_ATTN-GBO_GBO_GBO_PW,0.012054,0.018981,0.006812



FAILED PIPELINES:


,pipeline,test_rmse,decoder_rmse,problem_note
22,PCA_TRFM-PCA_PCA_AE_PW,0.054075,0.132380,TEMPORAL_FAIL
21,PCA_ATTN-PCA_PCA_AE_PW,0.054369,0.158082,TEMPORAL_FAIL
10,VAE_MLE_TRFM-V2-VAE_VAE_MLE_VAE_MLE_PW,0.103553,0.284230,TEMPORAL_FAIL
5,AE_TRFM-V2-AE_AE_AE_PW,0.128791,0.125019,TEMPORAL_FAIL
9,VAE_MLE_TRFM-VAE_VAE_MLE_VAE_MLE_PW,0.184451,0.153923,TEMPORAL_FAIL


In [ ]:
# ━━━━━━━━━━━━━ Add PCA-VAR Baselines to Results ━━━━━━━━━━━━━
from sklearn.decomposition import PCA
from statsmodels.tsa.api import VAR
from sklearn.metrics import root_mean_squared_error, mean_absolute_error  # ADD THIS
from utils.eval import summarize_latent_errors
from data.edav import check_no_arbitrage_total_variance

# Rest of your code continues...

def evaluate_pca_var_comprehensive(tensors, X_feat_tensor, test_split=0.8, augmented=False):
    """Comprehensive evaluation of PCA-VAR matching pipeline metrics"""
    
    # Extract surface tensor
    iv_surface = ops.convert_to_numpy(tensors["surface_tensor"])[..., -1]
    T, M, K = iv_surface.shape
    X_surface_flat = iv_surface.reshape(T, M * K).astype(np.float64)
    
    # Features
    F_aligned = ops.convert_to_numpy(X_feat_tensor).astype(np.float64)
    
    # Demean surfaces
    X_mean = X_surface_flat.mean(axis=1, keepdims=True)
    X_demeaned = X_surface_flat - X_mean
    
    # Fit PCA
    pca_model = PCA(n_components=LATENT_DIM)
    Z_all_pca = pca_model.fit_transform(X_demeaned)
    
    # Augment with features if requested
    if augmented:
        Z_aug = np.concatenate([Z_all_pca, F_aligned], axis=1)
    else:
        Z_aug = Z_all_pca
    
    # Train/test split
    T_train = int(test_split * T)
    T_test = T - T_train
    
    Z_train = Z_aug[:T_train]
    Z_test_true = Z_all_pca[T_train:]
    
    # Fit VAR
    var_model = VAR(Z_train)
    var_result = var_model.fit(maxlags=5)
    
    # Forecast
    Z_forecast_aug = var_result.forecast(Z_train[-var_result.k_ar:], steps=T_test)
    Z_forecast = Z_forecast_aug[:, :LATENT_DIM] if augmented else Z_forecast_aug
    
    # Latent space metrics
    latent_summary = summarize_latent_errors(
        Z_test_true, Z_forecast, 
        name=f"PCA_VAR_{'Augmented' if augmented else 'Surface_Only'}"
    )
    latent_metrics = latent_summary.iloc[0].to_dict()
    
    # Reconstruct surfaces
    X_recon_demeaned = pca_model.inverse_transform(Z_forecast)
    X_mean_test = X_mean[T_train:T_train + T_test]
    X_pred = X_recon_demeaned + X_mean_test
    X_true = X_surface_flat[T_train:T_train + T_test]
    
    # Reshape for evaluation
    X_pred_3d = X_pred.reshape(-1, M, K)
    X_true_3d = X_true.reshape(-1, M, K)
    
    # Decoder baseline
    X_decoder = pca_model.inverse_transform(Z_test_true) + X_mean_test
    decoder_rmse = root_mean_squared_error(X_true, X_decoder)
    
    # Persistence baseline
    X_prev = X_surface_flat[T_train-1:T_train-1+T_test]
    persistence_rmse = root_mean_squared_error(X_true, X_prev)
    
    # Train/test metrics
    test_split_idx = int(0.8 * T_test)
    X_test = X_true_3d[test_split_idx:]
    X_train_eval = X_true_3d[:test_split_idx]
    X_pred_test = X_pred_3d[test_split_idx:]
    X_pred_train = X_pred_3d[:test_split_idx]
    
    test_rmse = root_mean_squared_error(X_test.flatten(), X_pred_test.flatten())
    train_rmse = root_mean_squared_error(X_train_eval.flatten(), X_pred_train.flatten())
    full_rmse = root_mean_squared_error(X_true, X_pred)
    test_mae = mean_absolute_error(X_test.flatten(), X_pred_test.flatten())
    
    # Arbitrage violations
    rel_strikes = ops.convert_to_numpy(tensors["strike_tensor"])
    taus = ops.convert_to_numpy(tensors["taus"])
    
    smile_viol = 0
    cal_viol = 0
    for surface in X_pred_test:
        s, c = check_no_arbitrage_total_variance(surface, taus, rel_strikes, tol=0.005)
        smile_viol += s
        cal_viol += c
    
    return {
        'pipeline': f"PCA_VAR_{'Augmented' if augmented else 'Surface_Only'}",
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'full_rmse': full_rmse,
        'test_mae': test_mae,
        'decoder_rmse': decoder_rmse,
        'persistence_rmse': persistence_rmse,
        'latent_mse': latent_metrics['latent_mse_total'],
        'latent_acf_diff': latent_metrics['acf_diff_mean'],
        'latent_var_ratio': latent_metrics['var_ratio_pred/true'],
        'smile_violations': smile_viol,
        'calendar_violations': cal_viol,
        'total_violations': smile_viol + cal_viol,
        'is_problematic': False,
        'problem_note': ''
    }

# Append VAR results
print("\nEvaluation of  PCA-VAR baselines")
results.append(evaluate_pca_var_comprehensive(tensors, X_feat_tensor, augmented=False))
results.append(evaluate_pca_var_comprehensive(tensors, X_feat_tensor, augmented=True))

#  create DataFrame with everything but the problem pipelines 
df_results = pd.DataFrame(results)


Evaluation of  PCA-VAR baselines


In [22]:
def rank_pipelines(df_results, weights=None):
    """
    Rank pipelines considering RMSE vs practical fit quality trade-off.
    
    The "good RMSE, bad fit" problem means low error but poor arbitrage-free properties
    and poor temporal dynamics (ACF). This ranking balances accuracy with quality.
    
    Parameters:
    -----------
    df_results : DataFrame with pipeline evaluation metrics
    weights : dict with weight for each metric category (default: balanced)
    
    Returns:
    --------
    DataFrame with rankings and composite scores
    """
    
    if weights is None:
        weights = {
            'accuracy': 0.25,      # RMSE/MAE metrics
            'arbitrage': 0.50,     # No-arbitrage violations
            'temporal': 0.25,      # Latent temporal dynamics
            'generalization': 0.0  # Train/test gap
        }
    
    df = df_results.copy()
    
    # 1. Accuracy Score (lower is better → invert for scoring)
    df['accuracy_score'] = 1 / (1 + df['test_rmse'] / df['test_rmse'].min())
    df['accuracy_score_full'] = 1 / (1 + df['full_rmse'] / df['full_rmse'].min())
    
    # 2. Arbitrage Score (fewer violations is better)
    max_viol = df['total_violations'].max()
    df['arbitrage_score'] = 1 - (df['total_violations'] / (max_viol + 1))
    
    # 3. Temporal Dynamics Score (good ACF matching and variance preservation)
    df['temporal_score'] = (
        (1 / (1 + df['latent_acf_diff'])) * 0.5 +  # ACF matching
        (1 - abs(1 - df['latent_var_ratio'])) * 0.5  # Variance preservation
    )
    
    # 4. Generalization Score (small train/test gap)
    df['overfit_ratio'] = df['test_rmse'] / df['train_rmse']
    df['generalization_score'] = 1 / (1 + abs(df['overfit_ratio'] - 1))
    
    # ADD: Baseline comparisons
    df['improvement_over_decoder'] = (df['decoder_rmse'] - df['test_rmse']) / df['decoder_rmse']
    df['improvement_over_persistence'] = (df['persistence_rmse'] - df['test_rmse']) / df['persistence_rmse']
    
    # Composite Score (using test accuracy for main score)
    df['composite_score'] = (
        weights['accuracy'] * df['accuracy_score'] +
        weights['arbitrage'] * df['arbitrage_score'] +
        weights['temporal'] * df['temporal_score'] +
        weights['generalization'] * df['generalization_score']
    )
    
    # Rankings
    df['rank_test_rmse'] = df['test_rmse'].rank()
    df['rank_full_rmse'] = df['full_rmse'].rank()
    df['rank_arbitrage'] = df['total_violations'].rank()
    df['rank_temporal'] = df['latent_acf_diff'].rank()
    df['rank_overall'] = df['composite_score'].rank(ascending=False)
    
    # Quantify issues (0-1 scale instead of boolean)
    df['arbitrage_severity'] = df['total_violations'] / (df['total_violations'].max() + 1e-8)
    df['temporal_severity'] = df['latent_acf_diff'] / (df['latent_acf_diff'].max() + 1e-8)
    df['overfit_severity'] = np.maximum(0, (df['overfit_ratio'] - 1) / (df['overfit_ratio'].max() - 1 + 1e-8))
    
    # Final ranking with key metrics including baselines
    result = df[[
        'pipeline', 
        'test_rmse',
        'decoder_rmse',
        'persistence_rmse',
        'improvement_over_decoder',
        'improvement_over_persistence',
        'full_rmse',
        'total_violations',
        'latent_acf_diff',
        'composite_score',
        'rank_overall',
        'arbitrage_severity',
        'temporal_severity',
        'overfit_severity'
    ]].sort_values('rank_overall')
    
    # Round for display
    result = result.round(4)
    

    return result

# Apply ranking
ranked_results = rank_pipelines(df_results)
# remove duplicate pipelines
ranked_results = ranked_results[~ranked_results['pipeline'].duplicated()]

display(ranked_results)

,pipeline,test_rmse,decoder_rmse,persistence_rmse,improvement_over_decoder,improvement_over_persistence,full_rmse,total_violations,latent_acf_diff,composite_score,rank_overall,arbitrage_severity,temporal_severity,overfit_severity
6,VAE_GRU-VAE_VAE_VAE_PW,0.0107,0.0265,0.0068,0.5968,-0.5662,0.0076,656,0.0048,0.7970,1.0,0.1323,0.0104,0.3055
3,AE_GRU-AE_AE_AE_PW,0.0092,0.0218,0.0068,0.5773,-0.3539,0.0078,929,0.0027,0.7788,2.0,0.1873,0.0059,0.1231
2,AE_LSTM-ATTN-AE_AE_AE_PW,0.0091,0.0220,0.0068,0.5847,-0.3423,0.0079,926,0.0062,0.7781,3.0,0.1867,0.0135,0.1074
1,AE_LSTM-AE_AE_AE_PW,0.0092,0.0214,0.0068,0.5686,-0.3566,0.0078,934,0.0060,0.7755,4.0,0.1883,0.0131,0.1243
12,GBO_GRU-GBO_GBO_GBO_PW,0.0125,0.0204,0.0068,0.3870,-0.8355,0.0068,919,0.0050,0.7583,5.0,0.1853,0.0110,1.0000
7,VAE_MLE_LSTM-VAE_VAE_MLE_VAE_MLE_PW,0.0105,0.0312,0.0068,0.6623,-0.5470,0.0093,822,0.1774,0.7568,6.0,0.1657,0.3862,0.0875
8,VAE_MLE_LSTM-ATTN-VAE_VAE_MLE_VAE_MLE_PW,0.0112,0.0249,0.0068,0.5502,-0.6435,0.0082,919,0.1850,0.7455,7.0,0.1853,0.4027,0.2772
11,VAE_MLE_ATTN-VAE_VAE_MLE_VAE_MLE_PW,0.0110,0.0254,0.0068,0.5651,-0.6216,0.0091,949,0.1857,0.7446,8.0,0.1913,0.4041,0.1524
27,PCA_VAR_Augmented,0.0325,0.0013,0.0066,-24.5751,-3.9096,0.0179,459,0.0287,0.7404,9.5,0.0925,0.0624,0.4276
0,AE_ATTN-AE_AE_AE_PW,0.0089,0.0218,0.0068,0.5919,-0.3085,0.0078,933,0.4594,0.7290,11.0,0.1881,1.0000,0.0923


In [21]:

# Comparison table showing different ranking criteria
comparison_df = pd.DataFrame({
    'Pipeline': df_results['pipeline'].values,
    'Test_RMSE': df_results['test_rmse'].values,
    'Full_RMSE': df_results['full_rmse'].values,
    'Violations': df_results['total_violations'].values,
    'Rank_RMSE': df_results['test_rmse'].rank().astype(int).values,
    'Rank_Arbitrage': df_results['total_violations'].rank().astype(int).values,
})

# Add overall rank from ranked_results
rank_overall_dict = ranked_results.set_index('pipeline')['rank_overall'].to_dict()
comparison_df['Rank_Overall'] = comparison_df['Pipeline'].map(rank_overall_dict).astype(int)

comparison_df = comparison_df.sort_values('Rank_Overall')
display(comparison_df)

,Pipeline,Test_RMSE,Full_RMSE,Violations,Rank_RMSE,Rank_Arbitrage,Rank_Overall
6,VAE_GRU-VAE_VAE_VAE_PW,0.010669,0.007645,656,6,5,1
3,AE_GRU-AE_AE_AE_PW,0.009222,0.007823,929,3,12,2
2,AE_LSTM-ATTN-AE_AE_AE_PW,0.009143,0.007898,926,2,11,3
1,AE_LSTM-AE_AE_AE_PW,0.009241,0.007828,934,4,14,4
12,GBO_GRU-GBO_GBO_GBO_PW,0.012503,0.006755,919,11,9,5
7,VAE_MLE_LSTM-VAE_VAE_MLE_VAE_MLE_PW,0.010537,0.009325,822,5,8,6
8,VAE_MLE_LSTM-ATTN-VAE_VAE_MLE_VAE_MLE_PW,0.011195,0.008199,919,8,9,7
11,VAE_MLE_ATTN-VAE_VAE_MLE_VAE_MLE_PW,0.011046,0.009073,949,7,15,8
27,PCA_VAR_Augmented,0.032456,0.017881,459,15,2,9
25,PCA_VAR_Augmented,0.032456,0.017881,459,15,2,9
